# Importing Training and Testing Data

In [1]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('data/train.csv')

X_t = train_data.drop(['category', 'ID'], axis=1)
y_t = train_data['category']

test_data = pd.read_csv('data/test.csv')

# Approach 2: Pipelining the process

In [2]:
import metaclass as mc
import pipeline_components as pc
import importlib
importlib.reload(mc)

<module 'metaclass' from '/Users/Atharv/Documents/uni/Semester 4/CSE342 - Statistical Machine Learning [SML]/Project/SML-Project/metaclass.py'>

In [3]:
# labels = ['Apple_Raw', 'Apple_Ripe', 'Banana_Raw', 'Banana_Ripe', 'Coconut_Raw',
#  'Coconut_Ripe', 'Guava_Raw', 'Guava_Ripe', 'Leeche_Raw', 'Leeche_Ripe',
#  'Mango_Raw', 'Mango_Ripe', 'Orange_Raw', 'Orange_Ripe', 'Papaya_Raw',
#  'Papaya_Ripe', 'Pomengranate_Raw', 'Pomengranate_Ripe', 'Strawberry_Raw',
#  'Strawberry_Ripe']

# Get the category labels
# labels = sorted(list(set(y_t)))

# # Convert category labels to numbers
# y_t_int = y_t.map(lambda x: labels.index(x))
# print(labels)


In [9]:

pipeline = mc.Pipeline(
    clustering_alg=None,
    dim_reduction_algs=[("pca", 119)],
    outlier_detection_alg=None,
    classification_alg="logistic",
    ensemble_algs=None,
)
pipeline.fit(X_t, y_t)
print("Pipeline done")
# print("Pipeline score: ", pipeline.score(X_t, y_t))
cv_scores = pipeline.cross_validate(X_t, y_t)

Standardizing data...
Currently at clustering: ('kmeans', 20)


/Users/Atharv/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Currently at dim reduction


ValueError: n_components=119 must be between 0 and min(n_samples, n_features)=15 with svd_solver='full'

In [ ]:
# cv_scores.sort()
# 100 pca, 1 lda, logistic, no ensemble
cv_scores

(0.7911623827835121, 0.032399689396669776)

In [7]:
pipeline.generate_submission(test_data)
